<h1>Sentiment Analysis </h1>
<img src='https://storage.googleapis.com/kaggle-datasets-images/1569305/2583418/b232f5dc57e9b774fc6eafa6635f0782/dataset-cover.png?t=2021-09-03-12-12-22' width=120 height=100 >


### Dataset overview
#### Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

#### Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

**url**:(https://www.kaggle.com/datasets/meetnagadia/amazon-kindle-book-review-for-sentiment-analysis)

In [7]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [47]:
df=pd.read_csv('../artifacts/all_kindle_review .csv')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0.1    12000 non-null  int64 
 1   Unnamed: 0      12000 non-null  int64 
 2   asin            12000 non-null  object
 3   helpful         12000 non-null  object
 4   rating          12000 non-null  int64 
 5   reviewText      12000 non-null  object
 6   reviewTime      12000 non-null  object
 7   reviewerID      12000 non-null  object
 8   reviewerName    11962 non-null  object
 9   summary         11998 non-null  object
 10  unixReviewTime  12000 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 1.0+ MB


In [49]:
df.isnull().sum()

Unnamed: 0.1       0
Unnamed: 0         0
asin               0
helpful            0
rating             0
reviewText         0
reviewTime         0
reviewerID         0
reviewerName      38
summary            2
unixReviewTime     0
dtype: int64

In [50]:
df=df[['reviewText','rating']]

In [51]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewText  12000 non-null  object
 1   rating      12000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 187.6+ KB


In [44]:
df['rating'].value_counts()/len(df)*100

rating
5    25.000000
4    25.000000
3    16.666667
2    16.666667
1    16.666667
Name: count, dtype: float64

In [45]:
# df['reviewText'].apply(lambda x: x.lower()).info()

In [54]:
df['reviewText']=df.apply(lambda row: row['reviewText'].lower(),axis=1)

In [55]:
df['reviewText']

0        jace rankin may be short, but he's nothing to ...
1        great short read.  i didn't want to put it dow...
2        i'll start by saying this is the first of four...
3        aggie is angela lansbury who carries pocketboo...
4        i did not expect this type of book to be in li...
                               ...                        
11995    valentine cupid is a vampire- jena and ian ano...
11996    i have read all seven books in this series. ap...
11997    this book really just wasn't my cuppa.  the si...
11998    tried to use it to charge my kindle, it didn't...
11999    taking instruction is a look into the often hi...
Name: reviewText, Length: 12000, dtype: object

In [23]:
import re

In [25]:
# def clean_text(text):
#     # Remove URLs
#     text = re.sub(r'http\S+|www\.\S+', '', text)

#     # Remove HTML tags
#     text = re.sub(r'<.*?>', '', text)

#     # Keep only a-z, A-Z, 0-9, and !
#     text = re.sub(r'[^a-zA-Z0-9!]+', ' ', text)

#     # Optionally, strip extra spaces
#     text = text.strip()

#     return text

# # Example usage
# input_text = input("Enter your text: ")
# cleaned_text = clean_text(input_text)
# print("Cleaned Text:", cleaned_text)


In [235]:
import re
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Text cleaning function (vectorized)
def cleaner_column(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    if isinstance(text, str):
            # Lowercase
            text = text.lower()
            # Remove URLs
            text = re.sub(r'http\S+|www\.\S+', '', text)
            # Remove HTML tags
            text = re.sub(r'<.*?>', '', text)
            # Keep only a-z, 0-9, and !
            text = re.sub(r'[^a-z0-9!]+', ' ', text)
            # Strip extra spaces
            text = text.strip()
            
            # Lemmatize and remove stopwords
            words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
            
            return ' '.join(words)
    else:
            return ' '
    
    # return np.array(cleaned_texts).reshape(-1,1)



In [236]:
np.vectorize(cleaner_column)

In [237]:
df['reviewText'].apply(np.vectorize(cleaner_column))

0        jace rankin may short nothing mess man hauled ...
1        great short read want put read one sitting sex...
2        start saying first four book expecting 34 conc...
3        aggie angela lansbury carry pocketbook instead...
4        expect type book library pleased find price right
                               ...                        
11995    valentine cupid vampire jena ian another vampi...
11996    read seven book series apocalyptic adventure o...
11997    book really cuppa situation man capturing woma...
11998    tried use charge kindle even register charging...
11999    taking instruction look often hidden world sex...
Name: reviewText, Length: 12000, dtype: object

In [238]:
# df_['reviewText']

In [208]:
import nltk

In [209]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aman0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [210]:
cleaner_column(['hello my name is aman!! 1'])

array([['hello name aman!! 1']], dtype='<U19')

In [ ]:
# wrd embedding
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer


In [304]:
def get_text_column(X):
    return X[:, 0]    # return the list as strings only
# [[str1],[str2],[str3]]-->[str1,str2,str3]

In [ ]:
# method 1: tfidf
review_text_pipe_1 = Pipeline(
    [
        ('clean', FunctionTransformer(func=np.vectorize(cleaner_column))), 
        ('get_text', FunctionTransformer(get_text_column, validate=False)),  # Extract cleaned text
        ('tfidf', TfidfVectorizer(max_features=500))
    ]
)
# The last function in pipeline should return 2-d array but the get text return 1-d array because required by tfidf and bow
trf1 = ColumnTransformer([
    ('pipe_review',review_text_pipe_1,['reviewText']),
], remainder='passthrough')

In [306]:
trf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipe_review',
                                 Pipeline(steps=[('clean',
                                                  FunctionTransformer(func=<numpy.vectorize object at 0x000001D00339A910>)),
                                                 ('get_text',
                                                  FunctionTransformer(func=<function get_text_column at 0x000001D006D36AC0>)),
                                                 ('tfidf',
                                                  TfidfVectorizer(max_features=500))]),
                                 ['reviewText'])])

In [307]:
# Independent and dependent features
X,y=df.drop(columns=['rating']),df['rating']

In [308]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [309]:
X_train.shape

(8400, 1)

In [310]:
# test ctf1
X_train_transformed=trf1.fit_transform(X_train)

In [311]:
trf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipe_review',
                                 Pipeline(steps=[('clean',
                                                  FunctionTransformer(func=<numpy.vectorize object at 0x000001D00339A910>)),
                                                 ('get_text',
                                                  FunctionTransformer(func=<function get_text_column at 0x000001D006D36AC0>)),
                                                 ('tfidf',
                                                  TfidfVectorizer(max_features=500))]),
                                 ['reviewText'])])

In [312]:
X_train

,reviewText
7729,this was different and new to me. it was a ver...
3546,revised 05-06-13: my initial inclination was t...
3527,"not a good book, could not even finish reading..."
10159,i love kallysten's stories and this one did no...
6959,the book has one slightly worried about the po...
...,...
11964,i downloaded this book before reading any revi...
5191,this was by far one of the hottest books i've ...
5390,even though this book was free i had some rese...
860,a little too mushy and &#34;must take care of ...


In [313]:
X_train_transformed.shape

(8400, 500)

In [321]:
print(trf1.named_transformers_['pipe_review'].named_steps['tfidf'].get_feature_names_out())

['34' '8217' '99' 'ability' 'able' 'absolutely' 'across' 'action'
 'actually' 'add' 'adult' 'adventure' 'age' 'ago' 'almost' 'alone' 'along'
 'alpha' 'already' 'also' 'although' 'always' 'amazing' 'amazon' 'another'
 'anyone' 'anything' 'around' 'attention' 'attraction' 'author' 'away'
 'back' 'bad' 'beautiful' 'became' 'become' 'bed' 'begin' 'beginning'
 'behind' 'believable' 'believe' 'best' 'better' 'big' 'bit' 'body' 'book'
 'boring' 'bought' 'boy' 'break' 'brother' 'buy' 'call' 'called' 'came'
 'care' 'case' 'certainly' 'chance' 'change' 'chapter' 'character' 'check'
 'child' 'collection' 'come' 'coming' 'complete' 'completely' 'content'
 'continue' 'could' 'couple' 'course' 'cover' 'cute' 'dark' 'daughter'
 'day' 'deal' 'death' 'decided' 'definitely' 'depth' 'description'
 'detail' 'developed' 'development' 'dialogue' 'different' 'difficult'
 'disappointed' 'done' 'dragon' 'dream' 'easily' 'easy' 'editing' 'either'
 'else' 'emotion' 'emotional' 'end' 'ended' 'ending' 'enjoy' 'enj

In [322]:
X_train_transformed

<8400x500 sparse matrix of type '<class 'numpy.float64'>'
	with 204251 stored elements in Compressed Sparse Row format>

In [319]:
X_train_transformed.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.23860175, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.35351996, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
# Check tfidf etc first without any pipeline or any fe without transformer to ensure correctness

In [ ]:
# TO prevent errors we can separately create a vectorizer now

In [326]:
# The code for tfidf and bow is similar just replace their name
from sklearn.naive_bayes import GaussianNB,MultinomialNB
gnb=GaussianNB()
gnb.fit(X_train_transformed.toarray(),y_train)
mnb=MultinomialNB()
mnb.fit(X_train_transformed.toarray(),y_train)

MultinomialNB()

In [328]:
X_test_transformed=trf1.transform(X_test).toarray()
y_pred1=gnb.predict(X_test_transformed)
y_pred2=mnb.predict(X_test_transformed)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred=y_pred1,y_true=y_test))
print(classification_report(y_pred=y_pred2,y_true=y_test))

              precision    recall  f1-score   support

           1       0.39      0.49      0.43       612
           2       0.34      0.32      0.33       632
           3       0.26      0.28      0.27       572
           4       0.39      0.29      0.33       888
           5       0.51      0.55      0.53       896

    accuracy                           0.39      3600
   macro avg       0.38      0.39      0.38      3600
weighted avg       0.39      0.39      0.39      3600

              precision    recall  f1-score   support

           1       0.53      0.49      0.51       612
           2       0.45      0.25      0.32       632
           3       0.40      0.15      0.22       572
           4       0.37      0.57      0.45       888
           5       0.54      0.67      0.60       896

    accuracy                           0.46      3600
   macro avg       0.46      0.43      0.42      3600
weighted avg       0.46      0.46      0.44      3600



In [330]:
# since word2 vec doesn't have transform and fit so we create custom word2vec
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models import Word2Vec
import numpy as np

class Word2VecVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=1):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        tokenized_corpus = [doc.split() for doc in X]  # Assuming X is 1D array of strings
        self.model = Word2Vec(
            sentences=tokenized_corpus, 
            vector_size=self.vector_size, 
            window=self.window, 
            min_count=self.min_count
        )
        return self

    def transform(self, X):
        result = []
        for doc in X:
            words = doc.split()
            word_vecs = []
            for word in words:
                if word in self.model.wv:
                    word_vecs.append(self.model.wv[word])
            if word_vecs:
                avg_vec = np.mean(word_vecs, axis=0)
            else:
                avg_vec = np.zeros(self.vector_size)
            result.append(avg_vec)
        return np.array(result)


In [338]:
# method 1: word2vec
review_text_pipe_2 = Pipeline(
    [
        ('clean', FunctionTransformer(func=np.vectorize(cleaner_column))), 
        ('get_text', FunctionTransformer(get_text_column, validate=False)),  # Extract cleaned text
        ('word2vec', Word2VecVectorizer())
    ]
)
# The last function in pipeline should return 2-d array but the get text return 1-d array because required by tfidf and bow
trf2 = ColumnTransformer([
    ('pipe_review',review_text_pipe_2,['reviewText']),
], remainder='passthrough')

In [339]:
trf2.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipe_review',
                                 Pipeline(steps=[('clean',
                                                  FunctionTransformer(func=<numpy.vectorize object at 0x000001D003502350>)),
                                                 ('get_text',
                                                  FunctionTransformer(func=<function get_text_column at 0x000001D006D36AC0>)),
                                                 ('word2vec',
                                                  Word2VecVectorizer())]),
                                 ['reviewText'])])

In [340]:
X_train_transformed2=trf2.transform(X_train)

In [342]:
X_train_transformed2.shape

(8400, 100)

In [ ]:
# now similar things